In [ ]:
print("Hello, Jupyter!")

## セットアップ

In [ ]:
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
from dotenv import load_dotenv  # .env ファイルを読み込むためのライブラリ
import os
import traceback  # エラーの詳細を出力するためのモジュール

# .env ファイルを読み込む
load_dotenv()

# Azure AD テナント ID、クライアント ID、クライアントシークレット
tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("CLIENT_ID")
print(client_id)
client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")  # 取得したいユーザーの ID を指定
room_id = os.getenv("ROOM_ID")  # 取得したい会議室の ID を指定

# 必要なスコープ
scopes = ['https://graph.microsoft.com/.default']

try:
    # ClientSecretCredential を使用して認証
    credential = ClientSecretCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
    )

    # Microsoft Graph クライアントを作成
    graph_client = GraphServiceClient(credential, scopes)
    print("Microsoft Graph クライアントの作成に成功しました。")
except Exception as e:
    print("Microsoft Graph クライアントの作成に失敗しました:")
    print({e})
    print("----------")
    traceback.print_exc()  # エラーの詳細を出力

## ユーザー情報の取得
https://learn.microsoft.com/ja-jp/graph/api/user-get?view=graph-rest-1.0&tabs=python#request

In [ ]:
import asyncio

# ユーザーを取得する非同期関数
async def fetch_user(user_id):
    result = await graph_client.users.by_user_id(user_id).get()
    print(result)
    print(result.display_name)
    print(result.mail)

# 非同期関数を実行
await fetch_user(user_id)

## ユーザー一覧の取得  
会議室もユーザーと同様に取得可能  
https://stackoverflow.com/questions/56288573/isresourceaccount-always-null  
`Is Resource Account` はまだ使えないため、これで判別は難しいかも。後述する会議室一覧取得を使用するといい。  
https://learn.microsoft.com/en-us/graph/api/resources/user?view=graph-rest-1.0#properties

In [ ]:
import asyncio

# ユーザー一覧を取得する非同期関数
async def fetch_users():
    users_response = await graph_client.users.get()  # UserCollectionResponse オブジェクトを取得
    users = users_response.value  # ユーザーリストは value プロパティに格納されている
    for user in users:
        print(f"Email: {user.mail}, ID: {user.id}, Display Name: {user.display_name}, Is Resource Account: {user.is_resource_account}")
        #print(user)

# 非同期関数を実行
await fetch_users()

## カレンダー取得

In [ ]:
import asyncio

# 特定のユーザーのカレンダーを取得する非同期関数
async def fetch_user_calendars(user_id):
    # by_user_id メソッドを使用して特定のユーザーを指定
    user_request = graph_client.users.by_user_id(user_id)
    calendars_response = await user_request.calendars.get()  # User のカレンダーを取得
    calendars = calendars_response.value  # カレンダーリストは value プロパティに格納されている
    for calendar in calendars:
        print(f"ID: {calendar.id}, Name: {calendar.name}")

# 非同期関数を実行
# 取得したいユーザーの ID を指定
await fetch_user_calendars(user_id)

## イベント取得  
https://learn.microsoft.com/ja-jp/graph/api/user-list-events?view=graph-rest-1.0&tabs=python#request

In [ ]:
import asyncio

async def get_user_events(user_id):
    #from msgraph.generated.users.item.events.events_request_builder import EventsRequestBuilder

    #query_params = EventsRequestBuilder.EventsRequestBuilderGetQueryParameters(
    #    select=["subject", "start", "end"],  # 適切なプロパティを指定
    #)

    # 特定のユーザーのイベントを取得
    user_events = graph_client.users.by_user_id(user_id).events
    result = await user_events.get()
    print(result)

    # subject の一覧を出力
    for event in result.value:
        print(event.subject, event.start, event.end, event.location, event.body)

# 非同期関数を実行
await get_user_events(user_id)

## イベント登録  
`transaction_id`を設定することによって重複登録対策可能 UUID等設定

In [ ]:
import asyncio
from datetime import datetime  # datetime モジュールをインポート

async def post_user_events(user_id):
    from msgraph.generated.users.item.events.events_request_builder import EventsRequestBuilder
    from kiota_abstractions.base_request_configuration import RequestConfiguration
    from msgraph.generated.models.event import Event
    from msgraph.generated.models.item_body import ItemBody
    from msgraph.generated.models.body_type import BodyType
    from msgraph.generated.models.date_time_time_zone import DateTimeTimeZone
    from msgraph.generated.models.location import Location
    from msgraph.generated.models.attendee import Attendee
    from msgraph.generated.models.email_address import EmailAddress
    from msgraph.generated.models.attendee_type import AttendeeType

    # 現在の日時を取得してフォーマット
    current_time = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

    request_body = Event(
        subject = f"{current_time}Let's go for lunch",  # subject に日時を追加
        body = ItemBody(
            content_type = BodyType.Html,
            content = "Does noon work for you?",
        ),
        start = DateTimeTimeZone(
            date_time = "2025-04-15T17:00:00",
            time_zone = "Pacific Standard Time",
        ),
        end = DateTimeTimeZone(
            date_time = "2025-04-15T19:00:00",
            time_zone = "Pacific Standard Time",
        ),
        location = Location(
            display_name = "Harry's Bar",
        ),
        allow_new_time_proposals = True,
        # transaction_id = "7E163156-7762-4BEB-A1C6-729EA81755A7",
    )

    request_configuration = RequestConfiguration()
    request_configuration.headers.add("Prefer", "outlook.timezone=\"Pacific Standard Time\"")

    # 特定のユーザーのイベントを登録
    # イベントを作成
    result = await graph_client.users.by_user_id(user_id).events.post(request_body)
    print(result)

# 非同期関数を実行
await post_user_events(user_id)

## 会議室一覧取得  
https://learn.microsoft.com/ja-jp/graph/api/place-list?view=graph-rest-1.0&tabs=python

In [ ]:
import asyncio

async def get_place():

    #result = await graph_client.places.graph_room_list.get()
    result = await graph_client.places.graph_room.get()
    print(result)
    for place in result.value:
        print(f"ID: {place.id}, Display Name: {place.display_name}, Email: {place.email_address}")

# 非同期関数を実行
await get_place()

## 会議室イベント取得

In [ ]:
import asyncio

async def get_user_events(user_id):
    # from msgraph import GraphServiceClient
    # To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python

    result = await graph_client.users.by_user_id(user_id).calendar.events.get()

    print(result)

    # イベントの詳細を出力
    for event in result.value:
        print(f"件名: {event.subject}")
        print(f"開始日時: {event.start.date_time} ({event.start.time_zone})")
        print(f"終了日時: {event.end.date_time} ({event.end.time_zone})")
        print(f"場所: {event.location.display_name}")
        print(f"参加者:")
        for attendee in event.attendees:
            print(f"  - {attendee.email_address.name} ({attendee.email_address.address})")
        print(f"重要度: {event.importance}")
        print(f"キャンセル済み: {event.is_cancelled}")
        print(f"詳細: {event.body.content}")
        print("-" * 40)

# 非同期関数を実行
# 取得したい会議室の ID を指定
await get_user_events(room_id)

## 空き時間スケジュールを取得 
複数ユーザー、複数会議室指定  
https://learn.microsoft.com/ja-jp/graph/api/calendar-getschedule?view=graph-rest-1.0&tabs=python  

freeBusyStatus	出席者の空き時間の状態。 
https://learn.microsoft.com/ja-jp/graph/api/resources/attendeeavailability?view=graph-rest-1.0#properties  
使用可能な値: free、tentative、busy、oof、workingElsewhere、unknown。  
- free 空き時間
- tentative 仮の予定
- busy 予定あり
- oof 不在
- workingElsewhere 他の場所で仕事中
- unknown


In [ ]:
import asyncio

# メールアドレスを格納するリスト
email_addresses = []
# ユーザー取得の最大件数を指定する変数
max_users = 5  # 必要に応じて変更可能

# .env で指定した user_id のメールアドレスを取得
async def fetch_user(user_id):
    result = await graph_client.users.by_user_id(user_id).get()
    email_addresses.append(result.mail)
    print(result.mail)

# 会議室のメールアドレスを全件取得
async def get_place():
    result = await graph_client.places.graph_room.get()
    for place in result.value:
        email_addresses.append(place.email_address)
        print(place.email_address)

# ユーザー一覧を取得
async def fetch_users():
    users_response = await graph_client.users.get()  # UserCollectionResponse オブジェクトを取得
    users = users_response.value  # ユーザーリストは value プロパティに格納されている
    for user in users[:max_users]:  # 最初のmax_users件のみ取得
        email_addresses.append(user.mail)
        print(user.mail)

# スケジュールを取得
async def get_user_events(user_id):
    from msgraph.generated.users.item.calendar.get_schedule.get_schedule_post_request_body import GetSchedulePostRequestBody
    from msgraph.generated.models.date_time_time_zone import DateTimeTimeZone
    from kiota_abstractions.base_request_configuration import RequestConfiguration

    # スケジュールリクエストボディを作成
    request_body = GetSchedulePostRequestBody(
        schedules=email_addresses,  # 取得したメールアドレスを設定
        start_time=DateTimeTimeZone(
            date_time="2025-03-18T09:00:00",
            time_zone="Asia/Tokyo",
        ),
        end_time=DateTimeTimeZone(
            date_time="2025-04-30T18:00:00",
            time_zone="Asia/Tokyo",
        ),
        availability_view_interval=60,
    )

    request_configuration = RequestConfiguration()
    request_configuration.headers.add("Prefer", "outlook.timezone=\"Asia/Tokyo\"")

    result = await graph_client.users.by_user_id(user_id).calendar.get_schedule.post(
        request_body, request_configuration=request_configuration
    )

    # 結果を整形して表示
    print("スケジュール情報の取得結果:\n")
    for schedule in result.value:
        print(f"メールアドレス: {schedule.schedule_id}")
        if schedule.error:
            print(f"  - エラー: {schedule.error.message}")
            print(f"  - レスポンスコード: {schedule.error.response_code}")
        else:
            print("  - スケジュール情報:")
            if schedule.schedule_items:
                for item in schedule.schedule_items:
                    print(f"    - 開始: {item.start.date_time} ({item.start.time_zone})")
                    print(f"      終了: {item.end.date_time} ({item.end.time_zone})")
                    print(f"      状態: {item.status}")
            else:
                print("    スケジュール情報はありません。")
        print("\n")

# 非同期関数を実行
# 取得したいユーザーの ID を指定
await fetch_user(user_id)  # ユーザーのメールアドレスを取得
await get_place()          # 会議室のメールアドレスを取得
await fetch_users()        # ユーザー一覧からメールアドレスを取得
await get_user_events(user_id)  # スケジュールを取得

## メッセージを一覧表示する  
https://learn.microsoft.com/ja-jp/graph/api/user-list-messages?view=graph-rest-1.0&tabs=python#request

In [ ]:
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from kiota_abstractions.base_request_configuration import RequestConfiguration
# To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python

import asyncio

async def get_user_mails(user_id):
    query_params = MessagesRequestBuilder.MessagesRequestBuilderGetQueryParameters(
            select=["sender", "subject"],
    )

    request_configuration = RequestConfiguration(
        query_parameters=query_params,
    )

    result = await graph_client.users.by_user_id(user_id).messages.get(request_configuration=request_configuration)
    
    # ループで subject を出力
    for message in result.value:  # Assuming `result.value` contains the list of messages
        print(message.subject, message.sender)

# ユーザーIDを指定して関数を呼び出す
await get_user_mails(user_id)

## Web アプリケーションの作成
Flask を使用して、@メンション機能を持つ Web アプリケーションを実装します。

In [ ]:
# 必要なライブラリをインストール
# !pip install flask flask-cors

## シンプルな@メンション機能実装

より単純で確実に動作するバージョンの@メンションコンポーネントを実装します。

In [ ]:
from flask import Flask, jsonify, request, render_template_string
import threading
import json

# サンプルユーザーデータ (API呼び出しに依存しない)
SAMPLE_USERS = [
    {"id": "user1", "display_name": "田中太郎", "email": "tanaka@example.com"},
    {"id": "user2", "display_name": "山田花子", "email": "yamada@example.com"},
    {"id": "user3", "display_name": "佐藤一郎", "email": "sato@example.com"},
    {"id": "user4", "display_name": "鈴木次郎", "email": "suzuki@example.com"},
    {"id": "user5", "display_name": "高橋三郎", "email": "takahashi@example.com"},
    {"id": "user6", "display_name": "伊藤四郎", "email": "ito@example.com"},
    {"id": "user7", "display_name": "渡辺五郎", "email": "watanabe@example.com"},
    {"id": "user8", "display_name": "加藤六郎", "email": "kato@example.com"}
]

# シンプルなHTMLテンプレート
SIMPLE_HTML = '''
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>シンプルな@メンション検索</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        input {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            font-size: 16px;
        }
        .mention-wrapper {
            position: relative;
        }
        .mention-dropdown {
            position: absolute;
            top: 100%;
            left: 0;
            width: 100%;
            max-height: 200px;
            overflow-y: auto;
            background: white;
            border: 1px solid #ccc;
            border-radius: 4px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
            z-index: 1000;
            display: none;
        }
        .mention-item {
            padding: 8px 12px;
            cursor: pointer;
            border-bottom: 1px solid #eee;
        }
        .mention-item:hover {
            background-color: #f5f5f5;
        }
        .result {
            margin-top: 20px;
            padding: 15px;
            background-color: #f0f0f0;
            border-radius: 4px;
            min-height: 50px;
        }
        .mention {
            background-color: #e1f5fe;
            padding: 2px 4px;
            border-radius: 3px;
            font-weight: bold;
        }
        h1 {
            color: #333;
        }
        .status {
            margin-top: 10px;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            background-color: #f9f9f9;
        }
    </style>
</head>
<body>
    <h1>シンプルな@メンション検索</h1>
    <p>テキスト入力欄に「@」を入力すると、ユーザー一覧が表示されます。</p>
    
    <div class="mention-wrapper">
        <input type="text" id="mention-input" placeholder="ここに@を入力してユーザーを検索...">
        <div id="mention-dropdown" class="mention-dropdown"></div>
    </div>
    
    <button id="test-button">テスト: @を入力</button>
    
    <div class="status" id="status">ステータス: 準備完了</div>
    
    <h3>変換結果:</h3>
    <div class="result" id="result"></div>
    
    <script>
        // ユーザーデータ (サーバーサイドで定義したデータをインラインで使用)
        const users = ''' + json.dumps(SAMPLE_USERS, ensure_ascii=False) + ''';
        
        document.addEventListener('DOMContentLoaded', function() {
            const input = document.getElementById('mention-input');
            const dropdown = document.getElementById('mention-dropdown');
            const result = document.getElementById('result');
            const status = document.getElementById('status');
            const testButton = document.getElementById('test-button');
            
            let mentionMode = false;
            let mentionText = '';
            
            // ステータス更新
            function updateStatus(message) {
                status.textContent = 'ステータス: ' + message;
                console.log(message);
            }
            
            updateStatus('ページが読み込まれました');
            
            // テストボタン
            testButton.addEventListener('click', function() {
                input.value += '@';
                input.focus();
                checkForMention();
                updateStatus('テストボタンがクリックされました: @ が挿入されました');
            });
            
            // 入力フィールドの内容をチェック
            function checkForMention() {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @以降のテキストを取得
                    mentionText = text.substring(atIndex + 1);
                    mentionMode = true;
                    updateStatus('@が検出されました: 検索テキスト="' + mentionText + '"');
                    showUserDropdown(mentionText);
                } else {
                    mentionMode = false;
                    hideUserDropdown();
                }
            }
            
            // ユーザードロップダウンを表示
            function showUserDropdown(query) {
                // ドロップダウンをクリア
                dropdown.innerHTML = '';
                
                // ユーザーをフィルタリング
                const filteredUsers = users.filter(user => 
                    !query || user.display_name.toLowerCase().includes(query.toLowerCase())
                );
                
                if (filteredUsers.length === 0) {
                    // 該当ユーザーがない場合
                    const noResults = document.createElement('div');
                    noResults.className = 'mention-item';
                    noResults.textContent = '該当するユーザーがいません';
                    dropdown.appendChild(noResults);
                } else {
                    // ユーザー一覧を表示
                    filteredUsers.forEach(user => {
                        const item = document.createElement('div');
                        item.className = 'mention-item';
                        item.textContent = user.display_name;
                        item.dataset.id = user.id;
                        item.dataset.name = user.display_name;
                        
                        item.addEventListener('click', function() {
                            insertMention(user.display_name, user.id);
                        });
                        
                        dropdown.appendChild(item);
                    });
                }
                
                // ドロップダウンを表示
                dropdown.style.display = 'block';
                updateStatus('ドロップダウンを表示: ' + filteredUsers.length + ' 件');
            }
            
            // ユーザードロップダウンを非表示
            function hideUserDropdown() {
                dropdown.style.display = 'none';
            }
            
            // メンションを挿入
            function insertMention(name, id) {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @より前のテキストを保持し、@以降を置換
                    input.value = text.substring(0, atIndex) + '@' + name + ' ';
                    updateStatus('メンションを挿入しました: ' + name);
                    
                    // 結果に表示
                    const mentionSpan = document.createElement('span');
                    mentionSpan.className = 'mention';
                    mentionSpan.textContent = '@' + name;
                    mentionSpan.dataset.id = id;
                    
                    result.innerHTML = '';
                    result.appendChild(mentionSpan);
                    result.appendChild(document.createTextNode(' が選択されました'));
                }
                
                hideUserDropdown();
                mentionMode = false;
                input.focus();
            }
            
            // 入力イベントを監視
            input.addEventListener('input', function() {
                checkForMention();
            });
            
            // クリックイベントを監視（ドロップダウン外をクリックしたら閉じる）
            document.addEventListener('click', function(e) {
                if (!dropdown.contains(e.target) && e.target !== input && e.target !== testButton) {
                    hideUserDropdown();
                }
            });
            
            // ページロード時にステータスを更新
            updateStatus('読み込み完了 - ' + users.length + ' 人のユーザーデータが利用可能です');
        });
    </script>
</body>
</html>
'''

# シンプルなFlaskアプリケーション
simple_app = Flask(__name__)

@simple_app.route('/')
def simple_index():
    return render_template_string(SIMPLE_HTML)

# 別スレッドでFlaskアプリを実行
def run_simple_app():
    simple_app.run(host='0.0.0.0', port=5001, debug=False)

# シンプルなFlaskアプリケーションを起動
simple_flask_thread = threading.Thread(target=run_simple_app)
simple_flask_thread.daemon = True  # ノートブックが終了したらスレッドも終了
simple_flask_thread.start()

print("シンプルな@メンションアプリケーションを起動しました。http://localhost:5001 にアクセスしてください。")

## シンプル版 Web アプリケーションの使用方法

1. 上のセルを実行すると、新しい Flask サーバーが ポート 5001 で起動します
2. ブラウザで http://localhost:5001 にアクセスしてください
3. 以下の方法で @メンション機能をテストできます：
   - テキスト入力欄に直接 @ を入力する
   - 「テスト: @を入力」ボタンをクリックする
4. @ の後に文字を入力すると、ユーザー名がフィルタリングされます
5. 表示されたユーザー名をクリックすると、そのユーザー名が挿入されます

**このシンプル版の特徴**:
- Microsoft Graph API に依存せず、ローカルのサンプルデータを使用
- 通常の input フィールドを使用（contenteditable ではなく）
- エラーハンドリングとデバッグ出力の改善
- ステータス表示による視覚的フィードバック
- 単純なDOM操作とイベントハンドリング

**注意**: このアプリケーションはノートブックが実行中の間だけ利用可能です。

## Microsoft Graph APIを使用した@メンション機能

Microsoft Graph APIから取得した実際のユーザーデータを使用したシンプルな@メンション機能を実装します。

In [ ]:
from flask import Flask, jsonify, request, render_template_string
import threading
import json
import asyncio
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
from dotenv import load_dotenv
import os

load_dotenv()

# Azure AD 認証情報
tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
scopes = ['https://graph.microsoft.com/.default']

# Microsoft Graph APIからユーザーを非同期に取得する関数
async def fetch_graph_users():
    credential = ClientSecretCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
    )
    
    graph_client = GraphServiceClient(credential, scopes)
    users_response = await graph_client.users.get()
    return users_response.value

# 非同期関数を実行するためのヘルパー関数
def run_async_graph(coroutine):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        return loop.run_until_complete(coroutine)
    finally:
        loop.close()

# Microsoft Graph APIから取得したユーザーデータをキャッシュ
graph_users_cache = None

# フォールバック用のサンプルユーザーデータ
FALLBACK_USERS = [
    {"id": "user1", "display_name": "田中太郎", "email": "tanaka@example.com"},
    {"id": "user2", "display_name": "山田花子", "email": "yamada@example.com"},
    {"id": "user3", "display_name": "佐藤一郎", "email": "sato@example.com"}
]

# ユーザーデータを取得
def get_graph_users():
    global graph_users_cache
    
    # キャッシュがある場合はそれを使用
    if graph_users_cache:
        return graph_users_cache
    
    try:
        # Microsoft Graph APIからユーザーデータを取得
        users = run_async_graph(fetch_graph_users())
        
        # 必要なデータだけを抽出
        graph_users_cache = [
            {
                "id": user.id,
                "display_name": user.display_name or "Unknown User",
                "email": user.mail or ""
            } 
            for user in users if user.display_name and hasattr(user, 'id')
        ]
        
        print(f"Microsoft Graph APIから{len(graph_users_cache)}人のユーザーを取得しました")
        return graph_users_cache
    except Exception as e:
        print(f"Microsoft Graph APIからのユーザー取得エラー: {str(e)}")
        # エラーの場合はフォールバックデータを返す
        return FALLBACK_USERS

# シンプルなHTMLテンプレート
GRAPH_HTML = '''
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Microsoft Graph API @メンション検索</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        input {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            font-size: 16px;
        }
        .mention-wrapper {
            position: relative;
        }
        .mention-dropdown {
            position: absolute;
            top: 100%;
            left: 0;
            width: 100%;
            max-height: 200px;
            overflow-y: auto;
            background: white;
            border: 1px solid #ccc;
            border-radius: 4px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
            z-index: 1000;
            display: none;
        }
        .mention-item {
            padding: 8px 12px;
            cursor: pointer;
            border-bottom: 1px solid #eee;
        }
        .mention-item:hover {
            background-color: #f5f5f5;
        }
        .result {
            margin-top: 20px;
            padding: 15px;
            background-color: #f0f0f0;
            border-radius: 4px;
            min-height: 50px;
        }
        .mention {
            background-color: #e1f5fe;
            padding: 2px 4px;
            border-radius: 3px;
            font-weight: bold;
        }
        h1 {
            color: #333;
        }
        .status {
            margin-top: 10px;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            background-color: #f9f9f9;
        }
        .user-info {
            color: #666;
            font-size: 0.9em;
            margin-top: 3px;
        }
    </style>
</head>
<body>
    <h1>Microsoft Graph API @メンション検索</h1>
    <p>テキスト入力欄に「@」を入力すると、Microsoft Graph APIから取得したユーザー一覧が表示されます。</p>
    
    <div class="mention-wrapper">
        <input type="text" id="mention-input" placeholder="ここに@を入力してユーザーを検索...">
        <div id="mention-dropdown" class="mention-dropdown"></div>
    </div>
    
    <button id="test-button">テスト: @を挿入</button>
    
    <div class="status" id="status">ステータス: 準備完了</div>
    
    <h3>変換結果:</h3>
    <div class="result" id="result"></div>
    
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            const input = document.getElementById('mention-input');
            const dropdown = document.getElementById('mention-dropdown');
            const result = document.getElementById('result');
            const status = document.getElementById('status');
            const testButton = document.getElementById('test-button');
            
            let users = [];
            let mentionMode = false;
            let mentionText = '';
            
            // ステータス更新
            function updateStatus(message) {
                status.textContent = 'ステータス: ' + message;
                console.log(message);
            }
            
            updateStatus('データを読み込み中...');
            
            // Microsoft Graph APIからユーザーデータを取得
            fetch('/api/graph-users')
                .then(response => response.json())
                .then(data => {
                    users = data;
                    updateStatus('読み込み完了 - ' + users.length + ' 人のユーザーデータが利用可能です');
                    
                    if (users.length > 0) {
                        console.log('最初のユーザー:', users[0]);
                    }
                })
                .catch(error => {
                    updateStatus('エラー: ユーザーデータの読み込みに失敗しました');
                    console.error('エラー:', error);
                });
            
            // テストボタン
            testButton.addEventListener('click', function() {
                input.value += '@';
                input.focus();
                checkForMention();
                updateStatus('テストボタンがクリックされました: @ が挿入されました');
            });
            
            // 入力フィールドの内容をチェック
            function checkForMention() {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @以降のテキストを取得
                    mentionText = text.substring(atIndex + 1);
                    mentionMode = true;
                    updateStatus('@が検出されました: 検索テキスト="' + mentionText + '"');
                    showUserDropdown(mentionText);
                } else {
                    mentionMode = false;
                    hideUserDropdown();
                }
            }
            
            // ユーザードロップダウンを表示
            function showUserDropdown(query) {
                // ドロップダウンをクリア
                dropdown.innerHTML = '';
                
                // ユーザーをフィルタリング
                const filteredUsers = users.filter(user => 
                    !query || user.display_name.toLowerCase().includes(query.toLowerCase())
                );
                
                if (filteredUsers.length === 0) {
                    // 該当ユーザーがない場合
                    const noResults = document.createElement('div');
                    noResults.className = 'mention-item';
                    noResults.textContent = '該当するユーザーがいません';
                    dropdown.appendChild(noResults);
                } else {
                    // ユーザー一覧を表示 (最大10件まで)
                    const limitedUsers = filteredUsers.slice(0, 10);
                    limitedUsers.forEach(user => {
                        const item = document.createElement('div');
                        item.className = 'mention-item';
                        
                        // ユーザー名を表示
                        const nameDiv = document.createElement('div');
                        nameDiv.textContent = user.display_name;
                        item.appendChild(nameDiv);
                        
                        // メールアドレスを表示（あれば）
                        if (user.email) {
                            const emailDiv = document.createElement('div');
                            emailDiv.className = 'user-info';
                            emailDiv.textContent = user.email;
                            item.appendChild(emailDiv);
                        }
                        
                        item.dataset.id = user.id;
                        item.dataset.name = user.display_name;
                        
                        item.addEventListener('click', function() {
                            insertMention(user.display_name, user.id);
                        });
                        
                        dropdown.appendChild(item);
                    });
                    
                    // 件数表示が多すぎる場合は注釈を追加
                    if (filteredUsers.length > 10) {
                        const more = document.createElement('div');
                        more.className = 'mention-item';
                        more.style.textAlign = 'center';
                        more.style.fontStyle = 'italic';
                        more.textContent = `他に ${filteredUsers.length - 10} 人のユーザーがあります`;
                        dropdown.appendChild(more);
                    }
                }
                
                // ドロップダウンを表示
                dropdown.style.display = 'block';
                updateStatus('ドロップダウンを表示: ' + filteredUsers.length + ' 件');
            }
            
            // ユーザードロップダウンを非表示
            function hideUserDropdown() {
                dropdown.style.display = 'none';
            }
            
            // メンションを挿入
            function insertMention(name, id) {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @より前のテキストを保持し、@以降を置換
                    input.value = text.substring(0, atIndex) + '@' + name + ' ';
                    updateStatus('メンションを挿入しました: ' + name);
                    
                    // 結果に表示
                    const mentionSpan = document.createElement('span');
                    mentionSpan.className = 'mention';
                    mentionSpan.textContent = '@' + name;
                    mentionSpan.dataset.id = id;
                    
                    result.innerHTML = '';
                    result.appendChild(mentionSpan);
                    result.appendChild(document.createTextNode(' が選択されました'));
                }
                
                hideUserDropdown();
                mentionMode = false;
                input.focus();
            }
            
            // 入力イベントを監視
            input.addEventListener('input', function() {
                checkForMention();
            });
            
            // キーボードイベントも監視（特にスペースキーが押されたときなど）
            input.addEventListener('keyup', function(e) {
                checkForMention();
            });
            
            // クリックイベントを監視（ドロップダウン外をクリックしたら閉じる）
            document.addEventListener('click', function(e) {
                if (!dropdown.contains(e.target) && e.target !== input && e.target !== testButton) {
                    hideUserDropdown();
                }
            });
        });
    </script>
</body>
</html>
'''

# Microsoft Graph API連携バージョンのFlaskアプリケーション
graph_app = Flask(__name__)

@graph_app.route('/')
def graph_index():
    return render_template_string(GRAPH_HTML)

@graph_app.route('/api/graph-users')
def api_graph_users():
    users = get_graph_users()
    return jsonify(users)

# 別スレッドでFlaskアプリを実行
def run_graph_app():
    graph_app.run(host='0.0.0.0', port=5002, debug=False)

# Microsoft Graph API連携版アプリケーションを起動
graph_flask_thread = threading.Thread(target=run_graph_app)
graph_flask_thread.daemon = True  # ノートブックが終了したらスレッドも終了
graph_flask_thread.start()

print("Microsoft Graph API連携版@メンションアプリケーションを起動しました。http://localhost:5002 にアクセスしてください。")

## Microsoft Graph API連携版 Web アプリケーションの使用方法

1. 上のセルを実行すると、Microsoft Graph API連携版のFlaskサーバーがポート5002で起動します
2. ブラウザで http://localhost:5002 にアクセスしてください
3. 以下の方法で @メンション機能をテストできます：
   - テキスト入力欄に直接 @ を入力する
   - 「テスト: @を入力」ボタンをクリックする
4. @ の後に文字を入力すると、Microsoft Graph APIから取得したユーザー名がフィルタリングされます
5. 表示されたユーザー名をクリックすると、そのユーザー名が挿入されます

**この連携版の特徴**:
- Microsoft Graph APIから取得した実際のユーザーデータを使用
- ユーザー名に加えてメールアドレスも表示
- 最大10件のユーザーを表示（多すぎる場合は件数表示）
- API呼び出しエラー時はフォールバックデータを使用
- 簡単なキャッシュ機能によるパフォーマンス向上

**注意**: このアプリケーションはノートブックが実行中の間だけ利用可能です。

## ローカルキャッシュ機能付き@メンション

「最近使ったユーザー」をブラウザのローカルストレージに保存し、@のみ入力された場合はキャッシュからユーザーを表示する機能を実装します。
ユーザーが@に続けて文字を入力した場合は、通常通りすべてのユーザーから検索を行います。

In [ ]:
from flask import Flask, jsonify, request, render_template_string
import threading
import json
import asyncio
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
from dotenv import load_dotenv
import os

load_dotenv()

# Azure AD 認証情報
tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
scopes = ['https://graph.microsoft.com/.default']

# Microsoft Graph APIからユーザーを非同期に取得する関数
async def fetch_graph_users_cached():
    credential = ClientSecretCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
    )
    
    graph_client = GraphServiceClient(credential, scopes)
    users_response = await graph_client.users.get()
    return users_response.value

# 非同期関数を実行するためのヘルパー関数
def run_async_graph_cached(coroutine):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        return loop.run_until_complete(coroutine)
    finally:
        loop.close()

# Microsoft Graph APIから取得したユーザーデータをキャッシュ
graph_users_cache_local = None

# フォールバック用のサンプルユーザーデータ
FALLBACK_USERS_CACHED = [
    {"id": "user1", "display_name": "田中太郎", "email": "tanaka@example.com"},
    {"id": "user2", "display_name": "山田花子", "email": "yamada@example.com"},
    {"id": "user3", "display_name": "佐藤一郎", "email": "sato@example.com"}
]

# ユーザーデータを取得
def get_graph_users_cached():
    global graph_users_cache_local
    
    # キャッシュがある場合はそれを使用
    if (graph_users_cache_local):
        return graph_users_cache_local
    
    try:
        # Microsoft Graph APIからユーザーデータを取得
        users = run_async_graph_cached(fetch_graph_users_cached())
        
        # 必要なデータだけを抽出
        graph_users_cache_local = [
            {
                "id": user.id,
                "display_name": user.display_name or "Unknown User",
                "email": user.mail or ""
            } 
            for user in users if user.display_name and hasattr(user, 'id')
        ]
        
        print(f"Microsoft Graph APIから{len(graph_users_cache_local)}人のユーザーを取得しました")
        return graph_users_cache_local
    except Exception as e:
        print(f"Microsoft Graph APIからのユーザー取得エラー: {str(e)}")
        # エラーの場合はフォールバックデータを返す
        return FALLBACK_USERS_CACHED

# シンプルなHTMLテンプレート
CACHED_HTML = '''
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ローカルキャッシュ付き@メンション</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        input {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            font-size: 16px;
        }
        .mention-wrapper {
            position: relative;
        }
        .mention-dropdown {
            position: absolute;
            top: 100%;
            left: 0;
            width: 100%;
            max-height: 200px;
            overflow-y: auto;
            background: white;
            border: 1px solid #ccc;
            border-radius: 4px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
            z-index: 1000;
            display: none;
        }
        .mention-item {
            padding: 8px 12px;
            cursor: pointer;
            border-bottom: 1px solid #eee;
        }
        .mention-item:hover {
            background-color: #f5f5f5;
        }
        .mention-item.recent {
            background-color: #f0f8ff;
        }
        .mention-item.selected {
            background-color: #e0f0ff;
            border-left: 3px solid #4a86e8;
        }
        .result {
            margin-top: 20px;
            padding: 15px;
            background-color: #f0f0f0;
            border-radius: 4px;
            min-height: 50px;
        }
        .mention {
            background-color: #e1f5fe;
            padding: 2px 4px;
            border-radius: 3px;
            font-weight: bold;
        }
        h1 {
            color: #333;
        }
        .status {
            margin-top: 10px;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            background-color: #f9f9f9;
        }
        .user-info {
            color: #666;
            font-size: 0.9em;
            margin-top: 3px;
        }
        .source-label {
            font-size: 0.8em;
            margin-left: 10px;
            padding: 2px 4px;
            border-radius: 3px;
            background-color: #e0e0e0;
        }
        .source-label.recent {
            background-color: #c8e6c9;
            color: #2e7d32;
        }
        .controls {
            margin: 15px 0;
            display: flex;
            gap: 10px;
        }
        button {
            padding: 8px 12px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 4px;
            cursor: pointer;
        }
        button:hover {
            background-color: #45a049;
        }
        .keyboard-shortcut {
            margin-top: 10px;
            background-color: #f8f9fa;
            padding: 8px;
            border-radius: 4px;
            font-size: 0.9em;
        }
        .keyboard-shortcut kbd {
            background-color: #eee;
            border-radius: 3px;
            border: 1px solid #b4b4b4;
            box-shadow: 0 1px 1px rgba(0,0,0,0.2);
            color: #333;
            display: inline-block;
            font-size: 0.85em;
            padding: 2px 5px;
            margin: 0 2px;
        }
    </style>
</head>
<body>
    <h1>ローカルキャッシュ付き@メンション</h1>
    <p>テキスト入力欄に「@」のみを入力すると、<strong>最近使ったユーザー</strong>がローカルキャッシュから表示されます。</p>
    <p>「@」の後に文字を入力すると、通常通りすべてのユーザーから検索します。</p>
    
    <div class="keyboard-shortcut">
        <strong>キーボードショートカット:</strong>
        <ul>
            <li><kbd>↑</kbd>/<kbd>↓</kbd> キー: 候補間を移動</li>
            <li><kbd>Tab</kbd> または <kbd>Enter</kbd>: 選択中のユーザーを挿入</li>
            <li><kbd>Esc</kbd>: 候補を閉じる</li>
        </ul>
    </div>
    
    <div class="mention-wrapper">
        <input type="text" id="mention-input" placeholder="ここに@を入力してユーザーを検索...">
        <div id="mention-dropdown" class="mention-dropdown"></div>
    </div>
    
    <div class="controls">
        <button id="test-button">テスト: @を挿入</button>
        <button id="clear-history">履歴をクリア</button>
    </div>
    
    <div class="status" id="status">ステータス: 準備完了</div>
    
    <h3>変換結果:</h3>
    <div class="result" id="result"></div>
    
    <h3>最近使ったユーザー:</h3>
    <div class="result" id="recent-users">ユーザーを選択するとここに表示されます</div>
    
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            const input = document.getElementById('mention-input');
            const dropdown = document.getElementById('mention-dropdown');
            const result = document.getElementById('result');
            const recentUsersDiv = document.getElementById('recent-users');
            const status = document.getElementById('status');
            const testButton = document.getElementById('test-button');
            const clearHistoryButton = document.getElementById('clear-history');
            
            // ローカルストレージのキー
            const RECENT_USERS_KEY = 'recentMentionUsers';
            const MAX_RECENT_USERS = 5; // 最大履歴数
            
            let allUsers = [];
            let recentUsers = [];
            let mentionMode = false;
            let mentionText = '';
            let selectedIndex = -1; // 選択中のアイテムのインデックス
            
            // ステータス更新
            function updateStatus(message) {
                status.textContent = 'ステータス: ' + message;
                console.log(message);
            }
            
            // 最近使ったユーザーをローカルストレージから読み込む
            function loadRecentUsers() {
                try {
                    const storedUsers = localStorage.getItem(RECENT_USERS_KEY);
                    if (storedUsers) {
                        recentUsers = JSON.parse(storedUsers);
                        updateStatus('履歴から' + recentUsers.length + '人のユーザーを読み込みました');
                        displayRecentUsers();
                    } else {
                        updateStatus('履歴が見つかりません');
                    }
                } catch (e) {
                    console.error('履歴の読み込みエラー:', e);
                    updateStatus('履歴の読み込みに失敗しました');
                }
            }
            
            // 最近使ったユーザーを表示
            function displayRecentUsers() {
                recentUsersDiv.innerHTML = '';
                
                if (recentUsers.length === 0) {
                    recentUsersDiv.textContent = 'まだ履歴がありません';
                    return;
                }
                
                recentUsers.forEach(user => {
                    const userSpan = document.createElement('span');
                    userSpan.className = 'mention';
                    userSpan.textContent = '@' + user.display_name;
                    userSpan.title = user.email || '';
                    userSpan.style.marginRight = '10px';
                    recentUsersDiv.appendChild(userSpan);
                });
            }
            
            // 最近使ったユーザーに追加
            function addToRecentUsers(user) {
                // 既存の同じユーザーを削除
                recentUsers = recentUsers.filter(u => u.id !== user.id);
                
                // 先頭に追加
                recentUsers.unshift(user);
                
                // 最大数を超えたら古いものを削除
                if (recentUsers.length > MAX_RECENT_USERS) {
                    recentUsers = recentUsers.slice(0, MAX_RECENT_USERS);
                }
                
                // ローカルストレージに保存
                try {
                    localStorage.setItem(RECENT_USERS_KEY, JSON.stringify(recentUsers));
                    updateStatus('履歴に追加しました: ' + user.display_name);
                } catch (e) {
                    console.error('履歴の保存エラー:', e);
                    updateStatus('履歴の保存に失敗しました');
                }
                
                // 表示を更新
                displayRecentUsers();
            }
            
            // 履歴をクリア
            clearHistoryButton.addEventListener('click', function() {
                try {
                    localStorage.removeItem(RECENT_USERS_KEY);
                    recentUsers = [];
                    displayRecentUsers();
                    updateStatus('履歴をクリアしました');
                } catch (e) {
                    console.error('履歴のクリアエラー:', e);
                    updateStatus('履歴のクリアに失敗しました');
                }
            });
            
            updateStatus('データを読み込み中...');
            
            // 初期化処理
            loadRecentUsers();
            
            // Microsoft Graph APIからユーザーデータを取得
            fetch('/api/graph-users-cached')
                .then(response => response.json())
                .then(data => {
                    allUsers = data;
                    updateStatus('読み込み完了 - ' + allUsers.length + ' 人のユーザーデータが利用可能です');
                })
                .catch(error => {
                    updateStatus('エラー: ユーザーデータの読み込みに失敗しました');
                    console.error('エラー:', error);
                });
            
            // テストボタン
            testButton.addEventListener('click', function() {
                input.value += '@';
                input.focus();
                checkForMention();
                updateStatus('テストボタンがクリックされました: @ が挿入されました');
            });
            
            // ドロップダウンが表示されているかどうかを確認
            function isDropdownVisible() {
                return dropdown.style.display === 'block';
            }
            
            // 選択中のユーザーアイテムを取得
            function getSelectedUserItem() {
                if (!isDropdownVisible() || selectedIndex < 0) return null;
                
                const items = dropdown.querySelectorAll('.mention-item');
                if (selectedIndex >= items.length) return null;
                
                return items[selectedIndex];
            }
            
            // 選択されたユーザーの処理
            function processSelectedUser() {
                const selectedItem = getSelectedUserItem();
                if (selectedItem && selectedItem.dataset.id) {
                    // ユーザーが選択されていれば処理
                    const id = selectedItem.dataset.id;
                    const name = selectedItem.dataset.name;
                    
                    // ユーザーオブジェクトを探す
                    let userObj = allUsers.find(u => u.id === id);
                    if (!userObj) {
                        userObj = recentUsers.find(u => u.id === id);
                    }
                    
                    if (userObj) {
                        insertMention(name, id, userObj);
                        return true;
                    }
                }
                return false;
            }
            
            // 選択インデックスを更新して表示を反映
            function updateSelection(newIndex) {
                // 前の選択を解除
                const items = dropdown.querySelectorAll('.mention-item');
                items.forEach(item => item.classList.remove('selected'));
                
                // 新しい選択インデックスを設定
                selectedIndex = newIndex;
                
                // インデックスが有効範囲内なら選択表示
                if (selectedIndex >= 0 && selectedIndex < items.length) {
                    items[selectedIndex].classList.add('selected');
                    
                    // スクロールして選択項目が見えるようにする
                    items[selectedIndex].scrollIntoView({ block: 'nearest' });
                }
            }
            
            // キーボードイベントを監視（Tab, 矢印キー等）
            input.addEventListener('keydown', function(e) {
                if (!isDropdownVisible()) return;
                
                const items = dropdown.querySelectorAll('.mention-item');
                const itemCount = items.length;
                
                switch (e.key) {
                    case 'Tab':
                    case 'Enter':
                        // Tab または Enter キーで選択中のユーザーを挿入
                        if (processSelectedUser()) {
                            e.preventDefault(); // デフォルトの動作をキャンセル
                        }
                        break;
                        
                    case 'ArrowDown':
                        // 下矢印キーで次のユーザーを選択
                        e.preventDefault();
                        if (selectedIndex < 0) {
                            updateSelection(0); // 最初の項目を選択
                        } else {
                            updateSelection((selectedIndex + 1) % itemCount); // 次の項目へ（最後なら最初へ）
                        }
                        break;
                        
                    case 'ArrowUp':
                        // 上矢印キーで前のユーザーを選択
                        e.preventDefault();
                        if (selectedIndex < 0) {
                            updateSelection(itemCount - 1); // 最後の項目を選択
                        } else {
                            updateSelection((selectedIndex - 1 + itemCount) % itemCount); // 前の項目へ（最初なら最後へ）
                        }
                        break;
                        
                    case 'Escape':
                        // ESCキーでドロップダウンを閉じる
                        e.preventDefault();
                        hideUserDropdown();
                        break;
                }
            });
            
            // 入力フィールドの内容をチェック
            function checkForMention() {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @以降のテキストを取得
                    mentionText = text.substring(atIndex + 1);
                    mentionMode = true;
                    
                    if (mentionText === '') {
                        // @のみの場合、最近使ったユーザーを表示
                        updateStatus('@のみが検出されました: 最近使ったユーザーを表示');
                        showRecentUserDropdown();
                    } else {
                        // @の後に文字がある場合、通常の検索を実行
                        updateStatus('@が検出されました: 検索テキスト="' + mentionText + '"');
                        showUserDropdown(mentionText);
                    }
                } else {
                    mentionMode = false;
                    hideUserDropdown();
                }
            }
            
            // 最近使ったユーザーのドロップダウンを表示
            function showRecentUserDropdown() {
                // ドロップダウンをクリア
                dropdown.innerHTML = '';
                selectedIndex = -1; // 選択をリセット
                
                if (recentUsers.length === 0) {
                    // 履歴がない場合は通常の検索を実行
                    showUserDropdown('');
                    return;
                }
                
                // 履歴からユーザーを表示
                recentUsers.forEach(user => {
                    const item = document.createElement('div');
                    item.className = 'mention-item recent';
                    
                    // ユーザー名とラベルを表示
                    const nameDiv = document.createElement('div');
                    nameDiv.textContent = user.display_name;
                    
                    const sourceLabel = document.createElement('span');
                    sourceLabel.className = 'source-label recent';
                    sourceLabel.textContent = '最近';
                    nameDiv.appendChild(sourceLabel);
                    
                    item.appendChild(nameDiv);
                    
                    // メールアドレスを表示（あれば）
                    if (user.email) {
                        const emailDiv = document.createElement('div');
                        emailDiv.className = 'user-info';
                        emailDiv.textContent = user.email;
                        item.appendChild(emailDiv);
                    }
                    
                    item.dataset.id = user.id;
                    item.dataset.name = user.display_name;
                    
                    item.addEventListener('click', function() {
                        insertMention(user.display_name, user.id, user);
                    });
                    
                    dropdown.appendChild(item);
                });
                
                // すべてのユーザーを表示するオプション
                const showAllItem = document.createElement('div');
                showAllItem.className = 'mention-item';
                showAllItem.textContent = 'すべてのユーザーから検索...';
                showAllItem.style.textAlign = 'center';
                showAllItem.style.fontStyle = 'italic';
                showAllItem.addEventListener('click', function() {
                    showUserDropdown('');
                });
                dropdown.appendChild(showAllItem);
                
                // ドロップダウンを表示
                dropdown.style.display = 'block';
                updateStatus('最近使ったユーザーを表示: ' + recentUsers.length + ' 件');
                
                // 最初の項目を選択
                if (recentUsers.length > 0) {
                    updateSelection(0);
                }
            }
            
            // 通常のユーザードロップダウンを表示（すべてのユーザーから検索）
            function showUserDropdown(query) {
                // ドロップダウンをクリア
                dropdown.innerHTML = '';
                selectedIndex = -1; // 選択をリセット
                
                // ユーザーをフィルタリング
                const filteredUsers = allUsers.filter(user => 
                    !query || user.display_name.toLowerCase().includes(query.toLowerCase())
                );
                
                if (filteredUsers.length === 0) {
                    // 該当ユーザーがない場合
                    const noResults = document.createElement('div');
                    noResults.className = 'mention-item';
                    noResults.textContent = '該当するユーザーがいません';
                    dropdown.appendChild(noResults);
                } else {
                    // ユーザー一覧を表示 (最大10件まで)
                    const limitedUsers = filteredUsers.slice(0, 10);
                    limitedUsers.forEach(user => {
                        // 最近使ったユーザーかどうかをチェック
                        const isRecent = recentUsers.some(ru => ru.id === user.id);
                        
                        const item = document.createElement('div');
                        item.className = isRecent ? 'mention-item recent' : 'mention-item';
                        
                        // ユーザー名を表示
                        const nameDiv = document.createElement('div');
                        nameDiv.textContent = user.display_name;
                        
                        // 最近使ったユーザーならラベルを表示
                        if (isRecent) {
                            const sourceLabel = document.createElement('span');
                            sourceLabel.className = 'source-label recent';
                            sourceLabel.textContent = '最近';
                            nameDiv.appendChild(sourceLabel);
                        }
                        
                        item.appendChild(nameDiv);
                        
                        // メールアドレスを表示（あれば）
                        if (user.email) {
                            const emailDiv = document.createElement('div');
                            emailDiv.className = 'user-info';
                            emailDiv.textContent = user.email;
                            item.appendChild(emailDiv);
                        }
                        
                        item.dataset.id = user.id;
                        item.dataset.name = user.display_name;
                        
                        item.addEventListener('click', function() {
                            insertMention(user.display_name, user.id, user);
                        });
                        
                        dropdown.appendChild(item);
                    });
                    
                    // 件数表示が多すぎる場合は注釈を追加
                    if (filteredUsers.length > 10) {
                        const more = document.createElement('div');
                        more.className = 'mention-item';
                        more.style.textAlign = 'center';
                        more.style.fontStyle = 'italic';
                        more.textContent = `他に ${filteredUsers.length - 10} 人のユーザーがあります`;
                        dropdown.appendChild(more);
                    }
                    
                    // 最初の項目を選択
                    updateSelection(0);
                }
                
                // ドロップダウンを表示
                dropdown.style.display = 'block';
                updateStatus('ドロップダウンを表示: ' + filteredUsers.length + ' 件');
            }
            
            // ユーザードロップダウンを非表示
            function hideUserDropdown() {
                dropdown.style.display = 'none';
                selectedIndex = -1; // 選択をリセット
            }
            
            // メンションを挿入
            function insertMention(name, id, user) {
                const text = input.value;
                const atIndex = text.lastIndexOf('@');
                
                if (atIndex !== -1) {
                    // @より前のテキストを保持し、@以降を置換
                    input.value = text.substring(0, atIndex) + '@' + name + ' ';
                    updateStatus('メンションを挿入しました: ' + name);
                    
                    // 結果に表示
                    const mentionSpan = document.createElement('span');
                    mentionSpan.className = 'mention';
                    mentionSpan.textContent = '@' + name;
                    mentionSpan.dataset.id = id;
                    
                    result.innerHTML = '';
                    result.appendChild(mentionSpan);
                    result.appendChild(document.createTextNode(' が選択されました'));
                    
                    // 最近使ったユーザーに追加
                    if (user) {
                        addToRecentUsers(user);
                    }
                }
                
                hideUserDropdown();
                mentionMode = false;
                input.focus();
            }
            
            // 入力イベントを監視
            input.addEventListener('input', function() {
                checkForMention();
            });
            
            // キーボードイベントも監視（特にスペースキーが押されたときなど）
            input.addEventListener('keyup', function(e) {
                if (e.key !== 'Tab' && e.key !== 'Enter' && e.key !== 'ArrowUp' && e.key !== 'ArrowDown' && e.key !== 'Escape') {
                    checkForMention();
                }
            });
            
            // クリックイベントを監視（ドロップダウン外をクリックしたら閉じる）
            document.addEventListener('click', function(e) {
                if (!dropdown.contains(e.target) && e.target !== input && e.target !== testButton) {
                    hideUserDropdown();
                }
            });
        });
    </script>
</body>
</html>
'''

# ローカルキャッシュ版のFlaskアプリケーション
cached_app = Flask(__name__)

@cached_app.route('/')
def cached_index():
    return render_template_string(CACHED_HTML)

@cached_app.route('/api/graph-users-cached')
def api_graph_users_cached():
    users = get_graph_users_cached()
    return jsonify(users)

# 別スレッドでFlaskアプリを実行
def run_cached_app():
    cached_app.run(host='0.0.0.0', port=5003, debug=False)

# ローカルキャッシュ版アプリケーションを起動
cached_flask_thread = threading.Thread(target=run_cached_app)
cached_flask_thread.daemon = True  # ノートブックが終了したらスレッドも終了
cached_flask_thread.start()

print("ローカルキャッシュ付き@メンションアプリケーションを起動しました。http://localhost:5003 にアクセスしてください。")

## ローカルキャッシュ付き Web アプリケーションの使用方法

1. 上のセルを実行すると、ローカルキャッシュ付きのFlaskサーバーがポート5003で起動します
2. ブラウザで http://localhost:5003 にアクセスしてください
3. 以下の方法で @メンション機能をテストできます：
   - テキスト入力欄に直接 @ を入力する
   - 「テスト: @を入力」ボタンをクリックする
4. **@のみ** を入力した場合：
   - ブラウザの **ローカルストレージに保存された最近使ったユーザー** が表示されます
   - 履歴がない場合は全ユーザーリストが表示されます
5. **@に続けて文字** を入力した場合：
   - 通常通り、すべてのユーザーから検索されます
   - 最近使ったユーザーは「最近」というラベル付きで表示されます
6. ユーザーを選択すると自動的に「最近使ったユーザー」に追加され、次回の@入力時に表示されます
7. 「履歴をクリア」ボタンで最近使ったユーザーの履歴をリセットできます

**この機能の特徴**:
- ブラウザの **localStorage** を使用して最近使ったユーザーを保存
- @のみの入力時は API 呼び出しなしでキャッシュから高速表示
- 最大5人までの最近使ったユーザーを記憶
- 視覚的なラベルで最近使ったユーザーを識別可能
- ブラウザを閉じても履歴は保持されます

**注意**: このアプリケーションはノートブックが実行中の間だけ利用可能です。